In [1]:
!pip install azure-ai-ml azure-identity pandas
!pip install azureml-fsspec azureml-core
!pip install python-dotenv
!pip install mltable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.6/405.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.7/255.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.0/282.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication
import mltable


# Authentication

In [3]:
from dotenv import load_dotenv
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls /content/drive/MyDrive/secret/

Mounted at /content/drive
Write_env_file.ipynb


In [4]:
file_path = '/content/drive/MyDrive/secret/.env'
print(os.path.exists(file_path))

load_dotenv(file_path)
# Access the variables
azure_apply_id = os.getenv('AZURE_APPLY_ID')
azure_password = os.getenv('AZURE_PASSWORD')
azure_tenant = os.getenv('AZURE_TENANT')

True


In [5]:
os.environ["AZURE_CLIENT_ID"] = azure_apply_id
os.environ["AZURE_TENANT_ID"] = azure_tenant
os.environ["AZURE_CLIENT_SECRET"] = azure_password

# Upload Data
By reading data from GitHub and writing it to a local CSV file, then uploading it to the data section in Azure AI | Machine Learning Studio.

## Upload by file

In [6]:
import os
import subprocess
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Data
from azure.identity import DefaultAzureCredential

In [27]:
# Step 1: Get data from the Git repository
import requests

# URL of the file
url = "https://raw.githubusercontent.com/pandapanda3/Dataset_Machine_Learning/main/Twitter/twitter.csv"
local_file_path = "twitter.csv"

# Download the file
response = requests.get(url)
with open(local_file_path, "wb") as file:
    file.write(response.content)


In [17]:
# Step 2: Set up Azure ML client with credentials
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="04c1b27c-fcd8-43ba-96a2-cfe04a58d0a5",
    resource_group_name="demo-ml",
    workspace_name="demo-ml"
)

In [19]:
# List datasets in your Azure ML workspace to confirm connection
datasets = ml_client.data.list()
for dataset in datasets:
    print(dataset.name)


wine_rating


In [28]:
# Step 3: Register the specific file as a dataset

# Create a data asset pointing to the target path in the datastore
data_asset = Data(
    name="twitter_data",  # Give a unique name for this dataset in Azure ML
    path=local_file_path,
    type="uri_file",
    description="The dataset of twitter",
    datastore="workspaceblobstore",
    version="1"
)

In [29]:
# Step 4: Upload the data asset to Azure ML
ml_client.data.create_or_update(data_asset)

Uploading twitter.csv (< 1 MB): 100%|██████████| 3.10M/3.10M [00:00<00:00, 3.98MB/s]




Data({'path': 'azureml://subscriptions/04c1b27c-fcd8-43ba-96a2-cfe04a58d0a5/resourcegroups/demo-ml/workspaces/demo-ml/datastores/workspaceblobstore/paths/LocalUpload/450bc5c6c0589a0cc7fedc2c3b5aa3e5/twitter.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'twitter_data', 'description': 'The dataset of twitter', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/04c1b27c-fcd8-43ba-96a2-cfe04a58d0a5/resourceGroups/demo-ml/providers/Microsoft.MachineLearningServices/workspaces/demo-ml/data/twitter_data/versions/1', 'Resource__source_path': '', 'base_path': '/content', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7aa7a04e3ac0>, 'serialize': <msrest.serialization.Serializer object at 0x7aa7a04e2fb0>, 'version': '1', 'latest_version': None, 'datastore': None})

# Read a CSV file
The data is sourced from the dataset uploaded to Azure AI | Machine Learning Studio.





## mltable: Iterate quickly on a smaller subset of a large dataset
You might want to iterate quickly on a smaller subset of a large dataset before scaling up to a remote asynchronous job. mltable provides in-built functionality to get samples of large data using the take_random_sample method:

The `csv_file` variable is derived from the Datastore URI, which is an Azure Blob Storage URI pointing to the file location within the linked datastore in Azure Machine Learning Studio.

In [30]:
csv_file = 'azureml://subscriptions/04c1b27c-fcd8-43ba-96a2-cfe04a58d0a5/resourcegroups/demo-ml/workspaces/demo-ml/datastores/workspaceblobstore/paths/LocalUpload/450bc5c6c0589a0cc7fedc2c3b5aa3e5/twitter.csv'

In [31]:
path = {
    'file': csv_file
}

In [32]:
tbl = mltable.from_delimited_files(paths=[path])
df = tbl.to_pandas_dataframe()


In [33]:
df.head()

,id,label,tweet
0,1,False,@user when a father is dysfunctional and is s...
1,2,False,@user @user thanks for #lyft credit i can't us...
2,3,False,bihday your majesty
3,4,False,#model i love u take with u all the time in ...
4,5,False,factsguide: society now #motivation
